In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import tensorflow as tf
import pmaw
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from dotenv import load_dotenv

from libs.prep_methods import articles_pull, subreddit_pull, keyword_filter, articles_vader_analyzer, reddit_vader_analyzer, daily_mean

In [2]:
# stockmarket_comments = subreddit_pull('stockmarket', limit = 10000, after = int(dt.datetime(2012, 1, 1, 0, 0).timestamp()), before = int(dt.datetime(2022, 6, 1, 0, 0).timestamp()))

# securityanalysis_comments = subreddit_pull('securityanalysis', limit = 10000000, after = int(dt.datetime(2012, 1, 1, 0, 0).timestamp()), before = int(dt.datetime(2022, 6, 1, 0, 0).timestamp()))

# algotrading_comments = subreddit_pull('algotrading', limit = 10000000, after = int(dt.datetime(2012, 1, 1, 0, 0).timestamp()), before = int(dt.datetime(2022, 6, 1, 0, 0).timestamp()))

# wallstreetbets_comments = subreddit_pull('wallstreetbets', limit = 10000000, after = int(dt.datetime(2012, 1, 1, 0, 0).timestamp()), before = int(dt.datetime(2022, 6, 1, 0, 0).timestamp()))

# stockmarket_comments.to_csv('./Data/Cleaned_Data/stockmarket_comments.csv')
# securityanalysis_comments.to_csv('../../sandbox/securityanalysis_comments_large.csv')
# algotrading_comments.to_csv('../../sandbox/algotrading_comments_large.csv')
# wallstreetbets_comments.to_csv('./wallstreetbets_comments_large.csv')

In [3]:
stock_data = pd.read_csv('./Data/Cleaned_Data/stock_data.csv', parse_dates = True, infer_datetime_format = True)
stock_data['date'] = pd.to_datetime(stock_data['date'], infer_datetime_format = True, errors = 'coerce')
stock_data = stock_data.set_index('date')
stock_data.index.name = None

In [4]:
stockmarket_comments = pd.read_csv('../../sandbox/stockmarket_comments_large.csv', lineterminator = '\n', parse_dates = True, infer_datetime_format = True)

In [5]:
keywords = {
'NFLX': ['NFLX', 'nflx', 'Netflix', 'netflix'],
'FB': ['FB', 'fb', 'Facebook', 'facebook'],
'UBER': ['UBER', 'uber', 'Uber'],
'MCHP': ['MCHP', 'mchp', 'Microchip Technology'],
'ABNB': ['ABNB', 'abnb', 'AirBnB', 'airbnb'],
'FANG': ['FANG', 'fang', 'Diamondback Energy', 'diamondback energy', 'Diamondback', 'diamondback'],
'MRO': ['MRO', 'mro', 'Marathon Oil', 'marathon oil'],
'DVN': ['DVN', 'dvn', 'Devon Energy', 'devon energy'],
'SPWR': ['SPWR', 'spwr', 'SunPower', 'Sunpower', 'sunpower'],
'REGI': ['REGI', 'regi', 'Renewable Energy Group', 'renewable energy group'],
'MTRX': ['MTRX', 'mtrx', 'McKinsey & Company', 'McKinsey & Co', 'Mckinsey & Co', 'McKinsey', 'Mckinsey', 'mckinsey'],
'BLK': ['BLK', 'blk', 'BlackRock', 'Blackrock', 'blackrock'],
'PYPL': ['PYPL', 'pypl', 'PayPal', 'Paypal', 'paypal'],
'MELI': ['MELI', 'meli', 'MercadoLibre', 'Mercadolibre', 'mercadolibre'],
'SOFI': ['SOFI', 'sofi', 'SoFi', 'Sofi']
}

In [6]:
# asset_dataframe_list = []
# for asset in keywords:

#     asset_prices_volume = stock_data[stock_data['ticker'] == asset].drop(columns = 'ticker')

#     asset_stockmarket_comments = keyword_filter(stockmarket_comments.fillna(''), keywords[asset])

#     asset_stockmarket_daily_sentiment = daily_mean(reddit_vader_analyzer('stockmarket', asset_stockmarket_comments)).ffill(axis = 0)
#     asset_stockmarket_daily_sentiment['datetime'] = pd.to_datetime(asset_stockmarket_daily_sentiment['datetime'], infer_datetime_format = True, errors = 'coerce')
#     asset_stockmarket_daily_sentiment = asset_stockmarket_daily_sentiment.set_index('datetime')
#     asset_stockmarket_daily_sentiment.index = asset_stockmarket_daily_sentiment.index.date

#     asset_dataframe = pd.concat([asset_stockmarket_daily_sentiment, asset_prices_volume], axis = 1)
#     asset_dataframe.dropna(inplace = True)
#     asset_dataframe.to_csv(f'./Data/Cleaned_Data/{asset}.csv')

#     asset_dataframe_list.append(asset_dataframe)

In [7]:
asset_dataframe_list = []
for asset in keywords:

    asset_dataframe = pd.read_csv(f'./Data/Cleaned_Data/{asset}.csv', parse_dates = True, infer_datetime_format = True)
    asset_dataframe = asset_dataframe.set_index('Unnamed: 0')
    asset_dataframe.index.name = None

    asset_dataframe_list.append(asset_dataframe)

In [8]:
NFLX = pd.DataFrame(asset_dataframe_list[0])
FB = pd.DataFrame(asset_dataframe_list[1])
UBER = pd.DataFrame(asset_dataframe_list[2])
MCHP = pd.DataFrame(asset_dataframe_list[3])
ABNB = pd.DataFrame(asset_dataframe_list[4])
FANG = pd.DataFrame(asset_dataframe_list[5])
MRO = pd.DataFrame(asset_dataframe_list[6])
DVN = pd.DataFrame(asset_dataframe_list[7])
SPWR = pd.DataFrame(asset_dataframe_list[8])
REGI = pd.DataFrame(asset_dataframe_list[9])
MTRX = pd.DataFrame(asset_dataframe_list[10])
BLK = pd.DataFrame(asset_dataframe_list[11])
PYPL = pd.DataFrame(asset_dataframe_list[12])
MELI = pd.DataFrame(asset_dataframe_list[13])
SOFI = pd.DataFrame(asset_dataframe_list[14])

In [9]:
NFLX

,stockmarket_compound_sentiment,stockmarket_positive_sentiment,stockmarket_neutral_sentiment,stockmarket_negative_sentiment,close,volume
2015-12-01,0.640500,0.061000,0.911000,0.028000,125.37,12550755.0
2015-12-02,0.441443,0.158714,0.793857,0.047571,128.93,24105712.0
2015-12-03,0.441443,0.158714,0.793857,0.047571,126.81,26644540.0
2015-12-04,0.910000,0.362000,0.638000,0.000000,130.93,20250411.0
2015-12-07,0.964300,0.266000,0.734000,0.000000,125.36,23351531.0
...,...,...,...,...,...,...
2022-05-24,-0.134350,0.085000,0.832500,0.083000,180.34,7670343.0
2022-05-25,-0.380950,0.016000,0.806250,0.177750,187.83,8488881.0
2022-05-26,0.042725,0.103000,0.803750,0.093250,191.40,8039904.0
2022-05-27,0.042725,0.103000,0.803750,0.093250,195.19,8567288.0
